# Выбор локации для скважины

## Оглавление<a id="heading"></a>
1. [Описание проекта и условные обозначения](#1)
2. [Открытие файла и ознакомление с данными](#2)
3. [Обучение и проверка модели](#3)
4. [Подготовка к расчету прибыли](#4)
5. [Расчет прибыли и рисков](#5)
6. [Вывод](#6)

##  1. Описание проекта и условные обозначения<a id="1"></a>
[Открытие файла и ознакомление с данными▼](#2) [Оглавление▲▲](#heading)

Компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо определить регион, где добыча принесёт наибольшую прибыль.


Условные обозначения:

id — уникальный идентификатор скважины;

f0, f1, f2 — три признака скважины ;

product — объём запасов в скважине (тыс. баррелей).


### Условия задачи:

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

Бюджет на разработку скважин в регионе — 10 млрд рублей.
При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## 2. Открытие файла и ознакомление с данными<a id='2'>
[Обучение и проверка модели▼](#3) [Оглавление▲▲](#heading)


In [16]:
# загружаем библиотеки, читаем и ознакамливаемся с данными
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
pd.options.mode.chained_assignment = None

try:
    geo_1=pd.read_csv('C:/Users/ASUS/Desktop/практикум/модуль 2/спринт 3. Машинное обучение в бизнесе/проект/geo_data_0.csv')
    geo_2=pd.read_csv('C:/Users/ASUS/Desktop/практикум/модуль 2/спринт 3. Машинное обучение в бизнесе/проект/geo_data_1.csv')
    geo_3=pd.read_csv('C:/Users/ASUS/Desktop/практикум/модуль 2/спринт 3. Машинное обучение в бизнесе/проект/geo_data_2.csv')
except:
    geo_1=pd.read_csv('/datasets/geo_data_0.csv')
    geo_2=pd.read_csv('/datasets/geo_data_1.csv')
    geo_3=pd.read_csv('/datasets/geo_data_2.csv')
    
display(geo_1.info(), geo_2.info(), geo_3.info())
print('\n\n\n')

display(geo_1.head(), geo_2.head(), geo_3.head())
print('\n\n\n')

display(geo_1.corr(), geo_2.corr(), geo_3.corr())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

None

None

None

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


### Данные качественные, пропусков нет, тип данных соответствует. В данных по второму региону есть признак, который сильно коррелирует с целевым признаком. 

##  3. Обучение и проверка модели<a id="3"></a>
[Подготовка к расчету прибыли▼](#4) [Оглавление▲▲](#heading)

In [17]:
# создаем функцию для разбития датасетов на выборки и масштабирование
def split_scaler(data):
    data=data.drop('id', axis=1)           # удаляем ненужный признак
    target = data['product']
    features = data.drop('product', axis=1)
    features_train, features_valid, target_train, target_valid=train_test_split(
    features, target, test_size=0.25, random_state=1)

    
    
    scaler=StandardScaler()
    scaler.fit(features_train)
    features_train=scaler.transform(features_train)
    features_valid=scaler.transform(features_valid)
    
    return (features_train, features_valid, target_train, target_valid)

features_train_1, features_valid_1, target_train_1, target_valid_1=split_scaler(geo_1)
features_train_2, features_valid_2, target_train_2, target_valid_2=split_scaler(geo_2)
features_train_3, features_valid_3, target_train_3, target_valid_3=split_scaler(geo_3)

# проверяем размерность полученных датасетов
print(features_train_1.shape, features_valid_1.shape, target_train_1.shape, target_valid_1.shape)
print(features_train_2.shape, features_valid_2.shape, target_train_2.shape, target_valid_2.shape)
print(features_train_3.shape, features_valid_3.shape, target_train_3.shape, target_valid_3.shape)


(75000, 3) (25000, 3) (75000,) (25000,)
(75000, 3) (25000, 3) (75000,) (25000,)
(75000, 3) (25000, 3) (75000,) (25000,)


In [18]:
# создаем функцию для модели и подсчета метрик
def model_pred(features_train, target_train, features_valid, target_valid):
    model=LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)

    predicted_valid=pd.Series(predicted_valid)

    mse=mean_squared_error(target_valid, predicted_valid)
    rmse=mse**0.5
    r2=r2_score(target_valid, predicted_valid)
    mean=predicted_valid.mean()
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2:', r2)
    print('Среднее целевого признака:', mean)
    print('Коэффициенты линейной регрессии:', model.coef_)
    print('\n\n\n')
    return (predicted_valid, mse, rmse, r2, mean)


predicted_valid_1, mse_1, rmse_1, r2_1, mean_1=model_pred(
                            features_train_1, target_train_1, 
                            features_valid_1, target_valid_1)

predicted_valid_2, mse_2, rmse_2, r2_2, mean_2=model_pred(
                            features_train_2, target_train_2, 
                            features_valid_2, target_valid_2)

predicted_valid_3, mse_3, rmse_3, r2_3, mean_3=model_pred(
                            features_train_3, target_train_3, 
                            features_valid_3, target_valid_3)


MSE: 1424.5028508043274
RMSE: 37.74258669996437
R2: 0.27728067218414654
Среднее целевого признака: 92.49262459838843
Коэффициенты линейной регрессии: [ 3.38842716 -7.08166895 21.39937379]




MSE: 0.7998396764372653
RMSE: 0.8943375629130564
R2: 0.9996221352766932
Среднее целевого признака: 69.12040524285581
Коэффициенты линейной регрессии: [-1.30014526 -0.10879862 45.89221468]




MSE: 1589.3546681022217
RMSE: 39.86671127773423
R2: 0.20003372664683905
Среднее целевого признака: 94.95683048585317
Коэффициенты линейной регрессии: [-0.02172286  0.03053833 19.95724609]






### Выводы: ошибка моделей в 1 и 3 регионах на первый взгляд незначительна, не сильно отличается друг от друга и работают чуть лучше среднего по региону. В дальнейшем при расчете точки безубыточности сравним ее с ошибкой модели и сделаем более однозначные выводы. Модель во 2 регионе практически не имеет ошибки и судя по коэф. r2 предсказывает результаты почти идеально. Думаю это все благодаря тому признаку, который сильно коррелировал с выходом конечной продукции. Кстати, именно признак f2 имеет наибольший вес во всех регионах. Осмелюсь предположить, что именно этот регион и покажет в итоге самые лучшие результаты. Что касается средних целевого признака, то они в 1 и 3 регионах практически идентичны и примерно на 30% больше среднего 2 региона. Однако, если из этих средних вычесть ошибку модели, то 2 регион будет выдавать итоговой продукции примерно на 15% больше, что учитывая точность этой модели, заслуживает особого внимания.

##  4. Подготовка к расчету прибыли<a id="4"></a>
[Расчет прибыли и рисков▼](#5) [Оглавление▲▲](#heading)

In [19]:
# сохраняем условия задачи в константы
# инвестиции в регион
INVESTMENT=10000000000

# количество разрабатываемых скважин в регионе
COUNT=200

# доход с единицы продукта  
PRICE=450000

# точка безубыточности для 1 скважины
BREAK_EVEN_POINT=INVESTMENT/COUNT/PRICE
print('Точка безубыточности скважины, тыс.бар.: {:.2f}'.format(BREAK_EVEN_POINT))

Точка безубыточности скважины, тыс.бар.: 111.11


### Вывод: Точка безубыточности составляет 111 тыс. баррелей. Сравнивая эту величину с ошибкой моделей можно сказать, что ошибки не такие уж и маленькие. В первом регионе модель ошибается на 34%, а в третьем на 35,8%. Только лишь модель второго региона ошибается на 0.8%. 

##  5. Расчет прибыли и рисков<a id="5"></a>
[Вывод▼](#6) [Оглавление▲▲](#heading)

In [20]:
predicted_valid_1.index=target_valid_1.index
predicted_valid_2.index=target_valid_2.index
predicted_valid_3.index=target_valid_3.index


# создаем функцию для расчета прибыли
def profit_oil_well(target_subsample, pred_subsample, COUNT):
    top=pred_subsample.sort_values(ascending=False)[:COUNT]
    sum_product=target_subsample[top.index].sum()
    profit=sum_product*PRICE-INVESTMENT
    return profit


# применяем технику bootstrap для нахождения распределения прибыли с 
# параметрами: 1000 выборок и 500 точек
def bootstrap_profit(target_valid, predicted_valid):
    
    state = np.random.RandomState(1)
    col=0
    values=[]
    for i in range(1000):
        target_subsample=target_valid.sample(n=500, replace=True, random_state=state)
        pred_subsample=predicted_valid[target_subsample.index]
        profit_count_well=profit_oil_well(target_subsample, pred_subsample, COUNT)
        values.append(profit_count_well)
        
        if profit_count_well<0:
            col+=1
               
    values=pd.Series(values)        
    
    upper=values.quantile(0.975)
    lower=values.quantile(0.025)
    loss=col/len(values)
    
    print('Верхняя граница 95% доверительного интервала: {:.0f}'.format(upper))
    print('Нижняя граница 95% доверительного интервала: {:.0f}'.format(lower))
    print('Средняя прибыль в регионе: {:.0f}'.format(values.mean()))
    print('Риск убытков в регионе: {:.2%}'.format(loss))
    return (lower, upper, values.mean(), loss)
    
print('Регион 1')
print('--------------------------------------------------------')
bootstrap_profit_1=bootstrap_profit(target_valid_1, predicted_valid_1)
print('\n\n')  
print('Регион 2')
print('--------------------------------------------------------')
bootstrap_profit_2=bootstrap_profit(target_valid_2, predicted_valid_2)
print('\n\n')  
print('Регион 3')
print('--------------------------------------------------------')
bootstrap_profit_3=bootstrap_profit(target_valid_3, predicted_valid_3)

Регион 1
--------------------------------------------------------
Верхняя граница 95% доверительного интервала: 1260929215
Нижняя граница 95% доверительного интервала: 32469340
Средняя прибыль в регионе: 633382680
Риск убытков в регионе: 1.40%



Регион 2
--------------------------------------------------------
Верхняя граница 95% доверительного интервала: 1211367536
Нижняя граница 95% доверительного интервала: 197875679
Средняя прибыль в регионе: 682736565
Риск убытков в регионе: 0.30%



Регион 3
--------------------------------------------------------
Верхняя граница 95% доверительного интервала: 1247310951
Нижняя граница 95% доверительного интервала: -4560536
Средняя прибыль в регионе: 602787506
Риск убытков в регионе: 2.70%


##  5. Вывод<a id="6"></a>
[Оглавление▲▲](#heading)

### Как и предполагалось изначально лучше всех себя показал 2 регион: у него и самая высокая средняя прибыль (более 682 млн) и в разы меньше риск убытков (0.3%), а 95% доверительный интревал лежит в пределах 197 млн-1211 млн. Первый регион также подходит для разработки: средняя прибыль 633 млн. при доверительном интервале 324 млн.-1260 млн. и риске убытков 1.4%. А вот третий регион мало того, что ушел за границу приемлемого риска убытков в 2.7%, так еще и нижняя граница доверительного интервала ушла в отрицательную зону и составила -4.5 млн.